In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# ls


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!git clone https://github.com/AnthonyF333/FaceLandmark_PFLD_UltraLight.git

Cloning into 'FaceLandmark_PFLD_UltraLight'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 198 (delta 13), reused 0 (delta 0), pack-reused 174
Receiving objects: 100% (198/198), 19.18 MiB | 17.26 MiB/s, done.
Resolving deltas: 100% (90/90), done.


In [3]:
cd /kaggle/working/FaceLandmark_PFLD_UltraLight

/kaggle/working/FaceLandmark_PFLD_UltraLight


In [4]:
!pip install -r requirement.txt

In [5]:
!cp -r /kaggle/input/wider-facial-landmarks-inthewild-wflw/WFLW ./data

In [6]:
cd data

/kaggle/working/FaceLandmark_PFLD_UltraLight/data


In [7]:
!python SetPreparation.py

/kaggle/working/FaceLandmark_PFLD_UltraLight/data/test_data
file: 100/2500
file: 200/2500
file: 300/2500
file: 400/2500
file: 500/2500
file: 600/2500
file: 700/2500
file: 800/2500
file: 900/2500
file: 1000/2500
file: 1100/2500
file: 1200/2500
file: 1300/2500
file: 1400/2500
file: 1500/2500
file: 1600/2500
file: 1700/2500
file: 1800/2500
file: 1900/2500
file: 2000/2500
file: 2100/2500
file: 2200/2500
file: 2300/2500
file: 2400/2500
file: 2500/2500
/kaggle/working/FaceLandmark_PFLD_UltraLight/data/train_data
file: 100/7500
file: 200/7500
file: 300/7500
file: 400/7500
file: 500/7500
file: 600/7500
file: 700/7500
file: 800/7500
file: 900/7500
file: 1000/7500
file: 1100/7500
file: 1200/7500
file: 1300/7500
file: 1400/7500
file: 1500/7500
file: 1600/7500
file: 1700/7500
file: 1800/7500
file: 1900/7500
file: 2000/7500
file: 2100/7500
file: 2200/7500
file: 2300/7500
file: 2400/7500
file: 2500/7500
file: 2600/7500
file: 2700/7500
file: 2800/7500
file: 2900/7500
file: 3000/7500
file: 3100/7500
f

In [8]:
cd /kaggle/working/FaceLandmark_PFLD_UltraLight

/kaggle/working/FaceLandmark_PFLD_UltraLight


In [9]:
f = open("config.py",'w')
text='''
from torchvision import transforms as trans
from easydict import EasyDict as edict
from pfld.utils import get_time
import os
import torch


def get_config():
    cfg = edict()
    cfg.SEED = 2020
    cfg.DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    cfg.TRANSFORM = trans.Compose([trans.ToTensor(),
                                   trans.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

    cfg.MODEL_TYPE = 'PFLD_Ultralight'  # [PFLD, PFLD_Ultralight, PFLD_Ultralight_Slim]
    cfg.INPUT_SIZE = [96, 96]
    cfg.WIDTH_FACTOR = 0.25
    cfg.LANDMARK_NUMBER = 98

    cfg.TRAIN_BATCH_SIZE = 256
    cfg.VAL_BATCH_SIZE = 8

    cfg.TRAIN_DATA_PATH = './data/train_data/list.txt'
    cfg.VAL_DATA_PATH = './data/test_data/list.txt'

    cfg.EPOCHES = 200
    cfg.LR = 1e-4
    cfg.WEIGHT_DECAY = 1e-6
    cfg.NUM_WORKERS = 8
    cfg.MILESTONES = [90, 140, 170]

    cfg.RESUME = False
    if cfg.RESUME:
        cfg.RESUME_MODEL_PATH = ''

    create_time = get_time()
    cfg.MODEL_PATH = './checkpoint/models/{}_{}_{}_{}/'.format(cfg.MODEL_TYPE, cfg.WIDTH_FACTOR, cfg.INPUT_SIZE[0], create_time)
    cfg.LOG_PATH = './checkpoint/log/{}_{}_{}_{}/'.format(cfg.MODEL_TYPE, cfg.WIDTH_FACTOR, cfg.INPUT_SIZE[0], create_time)
    cfg.LOGGER_PATH = os.path.join(cfg.MODEL_PATH, "train.log")
    if not os.path.exists(cfg.MODEL_PATH):
        os.makedirs(cfg.MODEL_PATH)

    return cfg
'''
f.write(text)
f.close()

In [10]:
f=open("train.py",'w')
text = '''
#!/usr/bin/env python3
# -*- coding:utf-8 -*-

import logging
import warnings
import random
import os
import torch
import numpy as np

from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

from config import get_config
from dataset.datasets import WLFWDatasets
from pfld.utils import init_weights, save_checkpoint, set_logger, write_cfg
from pfld.loss import LandmarkLoss
from test import compute_nme

from models.PFLD import PFLD
from models.PFLD_Ultralight import PFLD_Ultralight
from models.PFLD_Ultralight_Slim import PFLD_Ultralight_Slim


def train(model, train_dataloader, loss_fn, optimizer, cfg):
    losses = []
    model.train()
    with tqdm(total=len(train_dataloader)) as t:
        for img, landmark_gt in train_dataloader:
            img = img.to(cfg.DEVICE)
            landmark_gt = landmark_gt.to(cfg.DEVICE)
            landmark_pred = model(img)
            loss = loss_fn(landmark_gt, landmark_pred)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            losses.append(loss.cpu().detach().numpy())
            t.update()

    return np.mean(losses)


def validate(model, val_dataloader, loss_fn, cfg):
    model.eval()
    losses = []
    nme_list = []
    with torch.no_grad():
        for img, landmark_gt in val_dataloader:
            img = img.to(cfg.DEVICE)
            landmark_gt = landmark_gt.to(cfg.DEVICE)
            landmark_pred = model(img)
            loss = loss_fn(landmark_gt, landmark_pred)
            losses.append(loss.cpu().numpy())

            landmark_pred = landmark_pred.reshape(landmark_pred.shape[0], -1, 2).cpu().numpy()
            landmark_gt = landmark_gt.reshape(landmark_gt.shape[0], -1, 2).cpu().numpy()
            nme_temp = compute_nme(landmark_pred, landmark_gt)
            for item in nme_temp:
                nme_list.append(item)

    return np.mean(losses), np.mean(nme_list)


def main():
    cfg = get_config()

    SEED = cfg.SEED
    np.random.seed(SEED)
    random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

    warnings.filterwarnings("ignore")
    set_logger(cfg.LOGGER_PATH)
    write_cfg(logging, cfg)

    main_worker(cfg)


def main_worker(cfg):
    # ======= LOADING DATA ======= #
    logging.warning('=======>>>>>>> Loading Training and Validation Data')
    TRAIN_DATA_PATH = cfg.TRAIN_DATA_PATH
    VAL_DATA_PATH = cfg.VAL_DATA_PATH
    TRANSFORM = cfg.TRANSFORM

    train_dataset = WLFWDatasets(TRAIN_DATA_PATH, TRANSFORM)
    train_dataloader = DataLoader(train_dataset, batch_size=cfg.TRAIN_BATCH_SIZE, shuffle=True, num_workers=cfg.NUM_WORKERS, drop_last=False)

    val_dataset = WLFWDatasets(VAL_DATA_PATH, TRANSFORM)
    val_dataloader = DataLoader(val_dataset, batch_size=cfg.VAL_BATCH_SIZE, shuffle=False, num_workers=cfg.NUM_WORKERS)

    # ======= MODEL ======= #
    MODEL_DICT = {'PFLD': PFLD,
                  'PFLD_Ghost': PFLD_Ghost,
                  'PFLD_Ghost_Slim': PFLD_Ghost_Slim,
                  }
    MODEL_TYPE = cfg.MODEL_TYPE
    WIDTH_FACTOR = cfg.WIDTH_FACTOR
    INPUT_SIZE = cfg.INPUT_SIZE
    LANDMARK_NUMBER = cfg.LANDMARK_NUMBER
    model = MODEL_DICT[MODEL_TYPE](WIDTH_FACTOR, INPUT_SIZE[0], LANDMARK_NUMBER).to(cfg.DEVICE)
    # model.apply(init_weights)
    if cfg.RESUME:
        if os.path.isfile(cfg.RESUME_MODEL_PATH):
            model.load_state_dict(torch.load(cfg.RESUME_MODEL_PATH))
        else:
            logging.warning("MODEL: No Checkpoint Found at '{}".format(cfg.RESUME_MODEL_PATH))
    logging.warning('=======>>>>>>> {} Model Generated'.format(MODEL_TYPE))

    # ======= LOSS ======= #
    loss_fn = LandmarkLoss(LANDMARK_NUMBER)
    logging.warning('=======>>>>>>> Loss Function Generated')

    # ======= OPTIMIZER ======= #
    optimizer = torch.optim.Adam(
        [{'params': model.parameters()}],
        lr=cfg.LR,
        weight_decay=cfg.WEIGHT_DECAY)
    logging.warning('=======>>>>>>> Optimizer Generated')

    # ======= SCHEDULER ======= #
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=cfg.MILESTONES, gamma=0.1)
    logging.warning('=======>>>>>>> Scheduler Generated' + '\n')

    # ======= TENSORBOARDX WRITER ======= #
    writer = SummaryWriter(cfg.LOG_PATH)

    dummy_input = torch.rand(1, 3, INPUT_SIZE[0], INPUT_SIZE[1]).to(cfg.DEVICE)
    writer.add_graph(model, (dummy_input,))

    best_nme = float('inf')
    for epoch in range(1, cfg.EPOCHES + 1):
        logging.warning('Epoch {} Start'.format(epoch))
        train_loss = train(model, train_dataloader, loss_fn, optimizer, cfg)
        val_loss, val_nme = validate(model, val_dataloader, loss_fn, cfg)
        scheduler.step()

        if val_nme < best_nme:
            best_nme = val_nme
            save_checkpoint(cfg, model, extra='best')
            logging.info('Save best model')
        save_checkpoint(cfg, model, epoch)

        writer.add_scalar('Learning_Rate', optimizer.param_groups[0]['lr'], epoch)
        writer.add_scalar('Train_Loss', train_loss, epoch)
        writer.add_scalar('Val_Loss', val_loss, epoch)
        writer.add_scalar('Val_NME', val_nme, epoch)

        logging.info('Train_Loss: {}'.format(train_loss))
        logging.info('Val_Loss: {}'.format(val_loss))
        logging.info('Val_NME: {}'.format(val_nme) + '\n')

    save_checkpoint(cfg, model, extra='final')


if __name__ == "__main__":
    main()
'''
f.write(text)
f.close()

In [11]:
f=open("test.py",'w')
text = '''
import argparse
import time

import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import simps

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
from dataset.datasets import WLFWDatasets

from models.PFLD import PFLD
from models.PFLD_Ultralight import PFLD_Ultralight
from models.PFLD_Ultralight_Slim import PFLD_Ultralight_Slim

cudnn.benchmark = True
cudnn.determinstic = True
cudnn.enabled = True


def compute_nme(preds, target):
    """ preds/target:: numpy array, shape is (N, L, 2)
        N: batchsize L: num of landmark
    """
    N = preds.shape[0]
    L = preds.shape[1]
    rmse = np.zeros(N)

    for i in range(N):
        pts_pred, pts_gt = preds[i,], target[i,]
        if L == 19:  # aflw
            interocular = 34  # meta['box_size'][i]
        elif L == 29:  # cofw
            interocular = np.linalg.norm(pts_gt[8,] - pts_gt[9,])
        elif L == 68:  # 300w
            # interocular
            interocular = np.linalg.norm(pts_gt[36,] - pts_gt[45,])
        elif L == 98:
            interocular = np.linalg.norm(pts_gt[60,] - pts_gt[72,])
        else:
            print(L)
            raise ValueError('Number of landmarks is wrong')
        rmse[i] = np.sum(np.linalg.norm(pts_pred - pts_gt, axis=1)) / (interocular * L)

    return rmse


def compute_auc(errors, failureThreshold, step=0.0001, showCurve=True):
    nErrors = len(errors)
    xAxis = list(np.arange(0., failureThreshold + step, step))
    ced = [float(np.count_nonzero([errors <= x])) / nErrors for x in xAxis]

    AUC = simps(ced, x=xAxis) / failureThreshold
    failureRate = 1. - ced[-1]

    if showCurve:
        plt.plot(xAxis, ced)
        plt.show()

    return AUC, failureRate


def validate(model, wlfw_val_dataloader, args):
    model.eval()

    nme_list = []
    cost_time = []
    with torch.no_grad():
        idx = 0
        for img, landmark_gt in wlfw_val_dataloader:
            img = img.to(args.device)
            landmark_gt = landmark_gt.to(args.device)

            start_time = time.time()
            landmarks = model(img)
            cost_time.append(time.time() - start_time)

            landmarks = landmarks.cpu().numpy()
            landmarks = landmarks.reshape(landmarks.shape[0], -1, 2)
            landmark_gt = landmark_gt.reshape(landmark_gt.shape[0], -1, 2).cpu().numpy()

            if args.show_image:
                show_img = np.array(np.transpose((img[0] * 0.5 + 0.5).cpu().numpy(), (1, 2, 0)))
                show_img = (show_img * 255).astype(np.uint8)
                np.clip(show_img, 0, 255)

                pre_landmark = landmarks[0, :, :2] * [args.input_size, args.input_size]

                cv2.imwrite("xxx.jpg", show_img)
                img_clone = cv2.imread("xxx.jpg")

                for ptidx, (x, y) in enumerate(pre_landmark):
                    cv2.circle(img_clone, (int(x), int(y)), 1, (0, 0, 255), -1)
                cv2.imwrite("xx_{}.jpg".format(idx), img_clone)

            nme_temp = compute_nme(landmarks, landmark_gt[:, :, :2])
            for item in nme_temp:
                nme_list.append(item)
            idx += 1
        # nme
        print('nme: {:.4f}'.format(np.mean(nme_list)))
        # auc and failure rate
        failureThreshold = 0.1
        auc, failure_rate = compute_auc(nme_list, failureThreshold)
        print('auc @ {:.1f} failureThreshold: {:.4f}'.format(failureThreshold, auc))
        print('failure_rate: {:}'.format(failure_rate))
        # inference time
        print("inference_cost_time: {0:4f}".format(np.mean(cost_time)))


def main(args):
    MODEL_DICT = {'PFLD': PFLD,
                  'PFLD_Ghost': PFLD_Ghost,
                  'PFLD_Ghost_Slim': PFLD_Ghost_Slim,
                  }
    MODEL_TYPE = args.model_type
    WIDTH_FACTOR = args.width_factor
    INPUT_SIZE = args.input_size
    LANDMARK_NUMBER = args.landmark_number
    model = MODEL_DICT[MODEL_TYPE](WIDTH_FACTOR, INPUT_SIZE, LANDMARK_NUMBER).to(args.device)

    checkpoint = torch.load(args.model_path, map_location=args.device)
    model.load_state_dict(checkpoint)

    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    wlfw_val_dataset = WLFWDatasets(args.test_dataset, transform)
    wlfw_val_dataloader = DataLoader(wlfw_val_dataset, batch_size=1, shuffle=False, num_workers=8)

    validate(model, wlfw_val_dataloader, args)


def parse_args():
    parser = argparse.ArgumentParser(description='Testing')
    parser.add_argument('--model_type', default='PFLD_Ghost', type=str)
    parser.add_argument('--input_size', default=112, type=int)
    parser.add_argument('--width_factor', default=1, type=float)
    parser.add_argument('--landmark_number', default=98, type=int)
    parser.add_argument('--device', default='cuda', type=str)
    parser.add_argument('--model_path', default="./checkpoint/models/PFLD_Ghost_1_112_2020-08-29-08-49/pfld_ghost_best.pth", type=str)
    parser.add_argument('--test_dataset', default='../../data/face_alignment/test_data/list.txt', type=str)
    parser.add_argument('--show_image', default=True, type=bool)
    args = parser.parse_args()
    return args


if __name__ == "__main__":
    args = parse_args()
    main(args)
'''
f.write(text)
f.close()

In [12]:
!python train.py

  File "train.py", line 131
    logging.warning('=======>>>>>>> Scheduler Generated' + '
                                                           ^
SyntaxError: EOL while scanning string literal
